# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,14.34,79,27,1.79,NZ,1738817759
1,1,freetown,8.4840,-13.2299,25.84,78,40,2.06,SL,1738817760
2,2,perryville,37.7242,-89.8612,6.62,100,100,1.54,US,1738817762
3,3,petropavlovsk-kamchatsky,53.0452,158.6483,-1.67,47,75,4.00,RU,1738817763
4,4,ust'-kulom,61.6864,53.6902,-19.76,100,74,1.12,RU,1738817764


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Max Temp", "Humidity", "Cloudiness"]
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Max Temp,Cloudiness)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
vacation_cities_df = city_data_df.loc[(city_data_df["Max Temp"] < 32) & (city_data_df["Cloudiness"] < 25)].copy()

# Drop any rows with null values
vacation_cities_df = vacation_cities_df.dropna(how='any', axis=0)

# Display sample data
vacation_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,bethel,41.3712,-73.4140,-4.69,60,0,0.89,US,1738817765
6,6,hobyo,5.3505,48.5268,25.31,74,18,7.25,SO,1738817766
10,10,zakynthos,37.7914,20.8953,9.90,70,0,2.70,GR,1738817771
13,13,dalbandin,28.8947,64.4101,11.45,23,1,3.51,PK,1738817774
14,14,prince rupert,54.3161,-130.3201,1.83,61,0,5.14,CA,1738817802


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_cities_df[["City","Country","Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,bethel,US,41.3712,-73.4140,60,
6,hobyo,SO,5.3505,48.5268,74,
10,zakynthos,GR,37.7914,20.8953,70,
13,dalbandin,PK,28.8947,64.4101,23,
14,prince rupert,CA,54.3161,-130.3201,61,
...,...,...,...,...,...,...
555,karachi,PK,24.9056,67.0822,28,
561,dzhalagash,KZ,45.0833,64.6667,60,
564,glendive,US,47.1053,-104.7125,84,
566,labuan,MY,5.2767,115.2417,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
hobyo - nearest hotel: Iftin Hotel Hobyo
zakynthos - nearest hotel: Phoenix Hotel
dalbandin - nearest hotel: SF Al-dawood Restaurant
prince rupert - nearest hotel: Crest Hotel
iqaluit - nearest hotel: Frobisher Inn
northam - nearest hotel: Durrant House Hotel
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
demopolis - nearest hotel: No hotel found
port mathurin - nearest hotel: Escale Vacances
qaqortoq - nearest hotel: Hotel Qaqortoq
sorel-tracy - nearest hotel: Hôtel de la Rive
faya-largeau - nearest hotel: No hotel found
grand gaube - nearest hotel: Hotel Lux- Grand Gaube
adamstown - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
ixtapa - nearest hotel: Ma. Cristina
nagqu - nearest hotel: Naqu Hotel
perieni - nearest hotel: Hotel Moldova
tocoa - nearest hotel: Gran Hotel Yadaly
den chai - nearest hotel: Den Chai
shimoda - nearest hotel: 

,City,Country,Lat,Lng,Humidity,Hotel Name
5,bethel,US,41.3712,-73.4140,60,Hampton Inn Danbury
6,hobyo,SO,5.3505,48.5268,74,Iftin Hotel Hobyo
10,zakynthos,GR,37.7914,20.8953,70,Phoenix Hotel
13,dalbandin,PK,28.8947,64.4101,23,SF Al-dawood Restaurant
14,prince rupert,CA,54.3161,-130.3201,61,Crest Hotel
...,...,...,...,...,...,...
555,karachi,PK,24.9056,67.0822,28,Quetta Agha Hotel
561,dzhalagash,KZ,45.0833,64.6667,60,No hotel found
564,glendive,US,47.1053,-104.7125,84,La Quinta Inn & Suites Glendive
566,labuan,MY,5.2767,115.2417,83,OYO 89344 Labuan Avenue Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)